In [44]:
import sciopy as sc
import numpy as np
from sciopy.sciopy_dataclasses import SingleFrame#, ScioSpecMeasurementConfig
import struct
from dataclasses import dataclass

In [45]:
@dataclass
class ScioSpecMeasurementConfig:
    com_port: str
    burst_count: int
    n_el: int
    channel_group: list
    actual_sample: int
    s_path: str
    object: str

In [2]:
sc.configuration_02?

Signature:
sc.configuration_02(
    serial,
    cnf: sciopy.sciopy_dataclasses.ScioSpecMeasurementConfig,
) -> sciopy.sciopy_dataclasses.ScioSpecMeasurementConfig
Docstring:
Amplitude [mA]      1.0
Range [V]           +/-5
Framerate           1
Gain                1
Burst Count         5

Min f [Hz]          1000
Max f [Hz]          1000
Steps               1
Scale               LINEAR

Measure Mode        Single Ended
Switch Type         Reed Relais

Channel Group Wizart:

Num. of Inj.        16
Chanel Group        Skip 0
File:      c:\users\sciospeceit\appdata\roaming\python\python39\site-packages\sciopy\configurations.py
Type:      function


In [3]:
sc.configuration_04?

Signature:
sc.configuration_04(
    serial,
    cnf: sciopy.sciopy_dataclasses.ScioSpecMeasurementConfig,
) -> sciopy.sciopy_dataclasses.ScioSpecMeasurementConfig
Docstring:
Amplitude [mA]      1.0
Range [V]           +/-5
Framerate           5
Gain                1
Burst Count         100

Min f [Hz]          15_000
Max f [Hz]          15_000
Steps               1
Scale               LINEAR

Measure Mode        Single Ended
Switch Type         Reed Relais

Channel Group Wizart:

Num. of Inj.        32
Chanel Group        Skip 0
File:      c:\users\sciospeceit\appdata\roaming\python\python39\site-packages\sciopy\configurations.py
Type:      function


In [63]:
tmp_16 = np.load('sciospeceit/pyender/tmp_data/meas_ele_16_bc_5.npy', allow_pickle=True)
tmp_32 = np.load('sciospeceit/pyender/tmp_data/meas_ele_32_bc_5.npy', allow_pickle=True)

In [64]:
print(tmp_16.shape)
print(tmp_32.shape)

(44804,)
(89604,)


In [65]:
sc_m_cnf = ScioSpecMeasurementConfig('COM3', burst_count=5, channel_group=[1], n_el=16, actual_sample=0, s_path='', object='circle')
sc_m_cnf.burst_count

5

In [70]:
def reshape_full_message_in_bursts(lst: list, cnf: ScioSpecMeasurementConfig)->np.ndarray:
    """
    Gets the full message buffer.
    
    Example:- input: n_el=16 -> lst.shape=(44804) | n_el=32 -> lst.shape=(89604,)
            - delete acknowledgement message: lst.shape=(4480,0) | lst.shape=(89600,)
            - split this depending on burst count: split_list.shape=(5, 8960) | split_list.shape=(5, 17920)
    """
    split_list = []
    msg_len = 140
    # delete acknowledgement message
    lst = lst[4:]
    # split in burst count messages
    split_length = lst.shape[0]//cnf.burst_count
    for split in range(cnf.burst_count):
        split_list.append(lst[split*split_length:(split+1)*split_length])
    return np.array(split_list)


from sciopy import parse_single_frame

def split_bursts_in_framge(split_list:np.ndarray, cnf : ScioSpecMeasurementConfig)->np.ndarray:
    """
    
    """
    msg_len = 140 # Constant
    frame = [] # Channel group depending frame
    burst_frame = [] # single burst count frame with channel depending frame
    subframe_length = split_list.shape[1]//msg_len
    for bursts in range(cnf.burst_count): # Iterate over bursts 
        tmp_split_list = np.reshape(split_list[bursts],(subframe_length,msg_len))
        for subframe in range(subframe_length):
            parsed_sgl_frame = parse_single_frame(tmp_split_list[subframe])
            # Select the right channel group data
            if parsed_sgl_frame.channel_group in cnf.channel_group:
                frame.append(parsed_sgl_frame)
        burst_frame.append(frame)
        frame = [] # Reset channel depending single burst frame
    return np.array(burst_frame) 

In [71]:
sc_m_cnf

ScioSpecMeasurementConfig(com_port='COM3', burst_count=5, n_el=16, channel_group=[1], actual_sample=0, s_path='', object='circle')

In [96]:
return_16 = reshape_full_message_in_bursts(tmp_16, sc_m_cnf)
return_32 = reshape_full_message_in_bursts(tmp_32, sc_m_cnf)

In [97]:
sc_m_cnf.channel_group=[1]
return_16_2 = split_bursts_in_framge(return_16, sc_m_cnf)
sc_m_cnf.channel_group=[1,2]
sc_m_cnf.n_el=32
return_32_2 = split_bursts_in_framge(return_32, sc_m_cnf)

In [98]:
return_16_2.shape

(5, 16)

In [100]:
for ele in return_16_2:
    print(ele.shape)

(16,)
(16,)
(16,)
(16,)
(16,)


In [ ]:
# update computing of the channel group

In [87]:
for ele in return_16_2[0]:
    print(ele.excitation_stgs)

['1' '2']
['2' '3']
['3' '4']
['4' '5']
['5' '6']
['6' '7']
['7' '8']
['8' '9']
['9' 'a']
['a' 'b']
['b' 'c']
['c' 'd']
['d' 'e']
['e' 'f']
['f' '10']
['10' '1']


In [ ]:
ari = ['f']

In [95]:
0x0a

10